# Install Required Packages

In [ ]:
!pip install pyathena
!pip3 install -U sagemaker

  Using cached pyathena-3.2.0-py3-none-any.whl.metadata (85 kB)
  Using cached urllib3-2.0.7-py3-none-any.whl.metadata (6.6 kB)
Using cached pyathena-3.2.0-py3-none-any.whl (79 kB)
Using cached urllib3-2.0.7-py3-none-any.whl (124 kB)
  Attempting uninstall: urllib3
    Found existing installation: urllib3 2.1.0
    Uninstalling urllib3-2.1.0:
      Successfully uninstalled urllib3-2.1.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
distributed 2022.7.0 requires tornado<6.2,>=6.0.3, but you have tornado 6.4 which is incompatible.
sagemaker 2.199.0 requires urllib3<1.27, but you have urllib3 2.0.7 which is incompatible.

[notice] A new release of pip is available: 23.3.1 -> 23.3.2
[notice] To update, run: pip install --upgrade pip
  Using cached sagemaker-2.206.0-py3-none-any.whl.metadata (13 kB)
  Using cached urllib3-1.26.18-py2.py3-none-any.whl.metadata (48 

# Import Required Libraries

In [ ]:
import boto3 # aws sdk for python
#import cv2 # computer vision
import sagemaker # machine learning platform
#import matplotlib.pyplot as plt # plotting functions
import numpy as np # array manipulation
import os # operating system interfaces
import pandas as pd # python data analysis
#import random # pseudo-random numbers
import re # regular expressions
#import seaborn as sns # statistical graphics
#import time # time access and conversions
#from PIL import Image # python imaging library module
from pyathena import connect # athena client
#from sagemaker import get_execution_role # get execution role for notebook
#from sagemaker.feature_store.feature_group import FeatureGroup # featuregroup definition
#from sagemaker.session import Session # class to initialize sagemaker session
from time import gmtime, strftime, sleep # time-related functions

# Perform Prerequisites

In [ ]:
# establish sagemaker session, provide permissions
sess = sagemaker.Session()
bucket = sess.default_bucket()
role = sagemaker.get_execution_role()
region = boto3.Session().region_name

# create a boto3 session for the sagemaker service
sm = boto3.Session().client(service_name='sagemaker', region_name=region)

# client to make featurestore record calls
featurestore_runtime = boto3.Session().client(
    service_name="sagemaker-featurestore-runtime", region_name=region
)

# create boto3 session to establish feature store session
boto_session = boto3.Session(region_name=region)

# create featurestore session
feature_store_session = sagemaker.Session(
    boto_session=boto_session,
    sagemaker_client=sm,
    sagemaker_featurestore_runtime_client=featurestore_runtime,
)

# define prefixes for the safety data directory and featurestore
prefix_data = 'safety/data'
prefix_featurestore = 'safety-featurestore'

# print s3 locations
print('Data directory location:', f"s3://{bucket}/{prefix_data}")
print('FeatureStore directory location:', f"s3://{bucket}/{prefix_featurestore}\n")

# print current IAM role for notebook instance
role = sagemaker.get_execution_role()
print('Execution Role:', role)

Data directory location: s3://sagemaker-us-east-1-414754026690/safety/data
FeatureStore directory location: s3://sagemaker-us-east-1-414754026690/safety-featurestore

Execution Role: arn:aws:iam::414754026690:role/LabRole


# Data Preparation

## Query Catalog Data

In [ ]:
# define database name
database_name = 'safetydb'

# define table name
table_name_csv = 'catalog_csv'

# set s3 temporary staging directory
s3_staging_dir = "s3://{0}/athena/staging".format(bucket)

# define connection parameters
conn = connect(region_name=region, s3_staging_dir=s3_staging_dir)

# define sql query statement
statement = """SELECT * FROM {}.{}
    WHERE img_filename like '%.jpg'
    AND label_filename like '%.txt'
    LIMIT 25000""".format(
    database_name, table_name_csv
)

# print sql statement for review before executing
print('SQL query SELECT statement:\n', statement)

SQL query SELECT statement:
 SELECT * FROM safetydb.catalog_csv
    WHERE img_filename like '%.jpg'
    AND label_filename like '%.txt'
    LIMIT 25000


In [ ]:
# execute sql query and display results
df_catalog_query = pd.read_sql(statement, conn)
df_catalog_query.head(10)

/tmp/ipykernel_17/3145676702.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_catalog_query = pd.read_sql(statement, conn)


,sample_id,img_filename,label_filename
0,000001,000001.jpg,000001.txt
1,000002,000002.jpg,000002.txt
2,000003,000003.jpg,000003.txt
3,000004,000004.jpg,000004.txt
4,000005,000005.jpg,000005.txt
5,000006,000006.jpg,000006.txt
6,000007,000007.jpg,000007.txt
7,000008,000008.jpg,000008.txt
8,000009,000009.jpg,000009.txt
9,000010,000010.jpg,000010.txt


## Combine with FeatureGroup Data

### Get Batch Records from FeatureGroups

In [ ]:
# define featuregroup name patterns
image_feature_group_name_pattern = 'image-feature-group-'
label_feature_group_name_pattern = 'label-feature-group-'

# obtain latest version of featuregroup names, if multiples exist
all_image_feature_groups = sm.list_feature_groups(NameContains=image_feature_group_name, SortBy='CreationTime', SortOrder='Descending')
all_label_feature_groups = sm.list_feature_groups(NameContains=label_feature_group_name, SortBy='CreationTime', SortOrder='Descending')
image_feature_group_name = all_image_feature_groups['FeatureGroupSummaries'][0]['FeatureGroupName']
label_feature_group_name = all_label_feature_groups['FeatureGroupSummaries'][0]['FeatureGroupName']

# print featuregroup names
print('Image FeatureGroup Name:', image_feature_group_name)
print('Label FeatureGroup Name:', label_feature_group_name)

Image FeatureGroup Name: image-feature-group-03-04-13-19
Label FeatureGroup Name: label-feature-group-03-04-13-19


In [ ]:
# specify record identifiers (i.e., sample ids) from previous catalog query
record_identifiers_value = df_catalog_query['sample_id'].values.astype(str).tolist()

# query image featuregroup by using record_id as primary key
df_image_feature_group = featurestore_runtime.batch_get_record(
    Identifiers=[
        {
            'FeatureGroupName': image_feature_group_name,
            'RecordIdentifiersValueAsString':record_identifiers_value,
        }
    ]
)

# query label featuregroup by using record_id as primary key
df_label_feature_group = featurestore_runtime.batch_get_record(
    Identifiers=[
        {
            'FeatureGroupName': label_feature_group_name,
            'RecordIdentifiersValueAsString': record_identifiers_value,
        }
    ]
)

### Display Image FeatureGroup Records

In [ ]:
# create list of image records and feature names, exclude eventtime
image_records = [sample['Record'] for sample in df_image_feature_group['Records']]
image_feature_names = [feature['FeatureName'] for feature in image_records[0] if feature['FeatureName'] != 'EventTime']

# create list of image data
image_data = [image_feature_names]

# iterate through each record in image featuregroup
for record in image_records:

    # iterate through each feature in individual record
    image_data.append([feature['ValueAsString'] for feature in record if feature['FeatureName'] != 'EventTime'])

# create images dataframe
df_images = pd.DataFrame(image_data[1:], columns=image_data[0])

# display dataframe head
df_images.head()

,sample_id,img_format,img_mode,img_height,img_width
0,000012,JPEG,RGB,640,640
1,000006,JPEG,RGB,640,640
2,000016,JPEG,RGB,640,640
3,000002,JPEG,RGB,640,640
4,000011,JPEG,RGB,640,640


### Display Label FeatureGroup Records

In [ ]:
# create list of label records and feature names, exclude eventtime
label_records = [sample['Record'] for sample in df_label_feature_group['Records']]
label_feature_names = [feature['FeatureName'] for feature in label_records[0] if feature['FeatureName'] != 'EventTime']

# create list of label data
label_data = [label_feature_names]

# iterate through each record in label featuregroup
for record in label_records:

    # iterate through each feature in individual record
    label_data.append([feature['ValueAsString'] for feature in record if feature['FeatureName'] != 'EventTime'])

# create labels dataframe
df_labels = pd.DataFrame(label_data[1:], columns=label_data[0])

# display dataframe head
df_labels.head()

,sample_id,count_helmet,count_vest,count_head
0,000012,4,0,0
1,000006,0,0,5
2,000016,1,0,0
3,000011,13,0,0
4,000002,2,1,0


### Join Records

In [ ]:
# merge catalog query and images dataframes, then the resulting with labels dataframe
df_combined = pd.merge(df_catalog_query, df_images, on='sample_id')
df_combined = pd.merge(df_combined, df_labels, on='sample_id')

# display resulting datafrmae
df_combined.head()

,sample_id,img_filename,label_filename,img_format,img_mode,img_height,img_width,count_helmet,count_vest,count_head
0,000001,000001.jpg,000001.txt,JPEG,RGB,640,640,2,0,0
1,000002,000002.jpg,000002.txt,JPEG,RGB,640,640,2,1,0
2,000003,000003.jpg,000003.txt,JPEG,RGB,640,640,4,0,0
3,000004,000004.jpg,000004.txt,JPEG,RGB,640,640,0,0,5
4,000005,000005.jpg,000005.txt,JPEG,RGB,640,640,2,0,0


## Split Data

In [ ]:
# use a copy of dataframe, can manipulate if desired for experimentation
data = df_combined.copy()

# data split in four sets - training, validation, test, and batch inference
rand_split = np.random.rand(len(data))
train_list = rand_split < 0.4
val_list = (rand_split >= 0.4) & (rand_split < 0.5)
test_list = (rand_split >= 0.5) & (rand_split < 0.6)
batch_list = rand_split >= 0.6 # "production" data

# print data splits
print('Data Splits:')
print('------------')
print(f"Train :   {sum(train_list)} samples")
print(f"Val   :   {sum(val_list)} samples")
print(f"Test  :   {sum(test_list)} samples")
print(f"Batch :   {sum(batch_list)} samples")

Data Splits:
------------
Train :   6 samples
Val   :   1 samples
Test  :   2 samples
Batch :   11 samples


## Create Split Datasets in S3

In [ ]:
# define and print source s3 locations
s3_images_source = f"s3://{bucket}/{prefix_data}/images/"
s3_labels_source = f"s3://{bucket}/{prefix_data}/labels/"
print('Images source directory location:', s3_images_source)
print('Labels source directory location:', s3_labels_source, '\n')

# define and print destination s3 locations for data splits
s3_images_dest = f"s3://{bucket}/{prefix_data}/split/images/"
s3_labels_dest = f"s3://{bucket}/{prefix_data}/split/labels/"
print('Images split destination directory location:', s3_images_dest)
print('Labels split destination directory location:', s3_labels_dest)

Images source directory location: s3://sagemaker-us-east-1-414754026690/safety/data/images/
Labels source directory location: s3://sagemaker-us-east-1-414754026690/safety/data/labels/ 

Images split destination directory location: s3://sagemaker-us-east-1-414754026690/safety/data/split/images/
Labels split destination directory location: s3://sagemaker-us-east-1-414754026690/safety/data/split/labels/


In [ ]:
# define function to copy files for respective data splits to corresponding s3 destinations
# provide 'split_name' as either 'train', 'val', 'test', or 'batch'
# provide 'split_list' as either 'train_list', 'val_list', 'test_list', or 'batch_list'
def split_dataset(split_name, split_list):

    # iterate through each sample in split
    for index, sample in data[split_list].iterrows():

        # source/destination variables for individual sample
        cp_image_source = f"{s3_images_source}{sample['img_filename']}"
        cp_image_dest = f"{s3_images_dest}{split_name}/"
        cp_label_source = f"{s3_labels_source}{sample['label_filename']}"
        cp_label_dest = f"{s3_labels_dest}{split_name}/"

        # copy from source to destination
        !aws s3 cp $cp_image_source $cp_image_dest
        !aws s3 cp $cp_label_source $cp_label_dest

In [ ]:
# perform data copies
print('Beginning TRAIN data split copies.')
split_dataset(split_name='train', split_list=train_list)
print('Completed TRAIN data split copies.')

print('Beginning VAL data split copies.')
split_dataset(split_name='val', split_list=val_list)
print('Completed VAL data split copies.')

print('Beginning TEST data split copies.')
split_dataset(split_name='test', split_list=test_list)
print('Completed TEST data split copies.')

print('Beginning BATCH data split copies.')
split_dataset(split_name='batch', split_list=batch_list)
print('Completed BATCH data split copies.')

copy: s3://sagemaker-us-east-1-414754026690/safety/data/images/000002.jpg to s3://sagemaker-us-east-1-414754026690/safety/data/split/images/test/000002.jpg
copy: s3://sagemaker-us-east-1-414754026690/safety/data/labels/000002.txt to s3://sagemaker-us-east-1-414754026690/safety/data/split/labels/test/000002.txt


# Training Job and Model Creation

# TESTING BELOW...

In [ ]:
!pip install ultralytics

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.2/40.2 kB 507.8 kB/s eta 0:00:00a 0:00:01
  Using cached opencv_python-4.9.0.80-cp37-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (20 kB)
  Using cached typing_extensions-4.9.0-py3-none-any.whl.metadata (3.0 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 34.6 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 9.2 MB/s eta 0:00:0000:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 38.8 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 2.6 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 11.1 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 26.6 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 8.5 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0/196.0 MB 5.6 MB/s eta 0:00:0000:0100:01
 

In [ ]:
!pip install opencv-python-headless

  Using cached opencv_python_headless-4.9.0.80-cp37-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (20 kB)
Using cached opencv_python_headless-4.9.0.80-cp37-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (49.6 MB)

[notice] A new release of pip is available: 23.3.1 -> 23.3.2
[notice] To update, run: pip install --upgrade pip


# Create Estimator

In [ ]:
from sagemaker.pytorch import PyTorch
import cv2
from ultralytics import YOLO

pytorch_estimator = PyTorch('train.py',
                            instance_type='ml.m5.xlarge',
                            instance_count=1,
                            framework_version='1.8.0',
                            py_version='py3',
                            hyperparameters = {'epochs': 2, 'batch-size': 2, 'learning-rate': 0.1},
                            sagemaker_session=sm)

pytorch_estimator.fit({'train': "{}".format(s3_images_dest),
                       'test': "{}".format(s3_images_dest)})

AttributeError: 'SageMaker' object has no attribute 'local_mode'